## 과제 1
ReLu activation function과 derivative function을 구현해보세요
- Hint : np.maximum 함수 사용하면 편리합니다
- 다른 방법 사용하셔도 무방합니다


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def relu(x):
    out = np.maximum(0, x)
    return out

In [ ]:
def d_relu(x):
    if relu(x) != 0:
        out = 1
    else:
        out = 0
    return out

## 과제 2
Deep Learning Basic 코드 파일의 MLP implementation with Numpy library using MNIST dataset 코드 참고해서
Three layer MLP 일 때의 backward_pass 함수를 완성해주세요.   
- Hint : 코드 파일의 예시는 Two layer MLP


In [ ]:
def backward_pass(x, y_true, params):
  dS3 = params["A3"] - y_true

  grads = {}

  grads["dW3"] =  np.dot(dS3, params["A2"].T)/x.shape[1]
  grads["db3"] =  (1/x.shape[1])*np.sum(dS3, axis=1, keepdims=True)/x.shape[1]

  dA2 = np.dot(params["W3"].T, dS3)
  dS2 = dA2 * d_sigmoid(params["S2"])

  grads["dW2"] = np.dot(dS2, x.T)/x.shape[1]
  grads["db2"] = np.sum(dS2, axis=1, keepdims=True)/x.shape[1]

  dA1 = np.dot(params["W2"].T, dS2)
  dS1 = dA1 * d_sigmoid(params["S1"])

  grads["dW1"] = np.dot(dS1, x.T)/x.shape[1]
  grads["db1"] = np.sum(dS1, axis=1, keepdims=True)/x.shape[1] 

  return grads

## 과제 3
Deep Learning Basic 코드 파일의 MLP implementation with Pytorch library using MNIST dataset 코드 참고해서
Three layer MLP를 구한후, 학습을 돌려 보세요

hyperparameter는 다음과 같이 설정

- epochs : 100
- hiddensize : 128, 64 (two layer)
- learning_rate : 0.5

In [ ]:
from torchvision import transforms, datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
trainset = datasets.MNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.MNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/MNIST/raw



In [ ]:
BATCH_SIZE = 512
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader =  DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# Assignment 3 구현은 여기서 ()


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(784,128)
        self.layer2 = nn.Linear(128,64)
        self.layer3 = nn.Linear(64,10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)

        return out

In [ ]:
model = Net()
model

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [ ]:
def train(model, train_loader, optimizer):
    model.train()
    # 배치 당 loss 값을 담을 리스트 생성
    batch_losses = []

    for data, target in train_loader:
        # 옵티마이저의 기울기 초기화
        optimizer.zero_grad()

        # y pred 값 산출
        output = model(data)
        # loss 계산
        # 정답 데이터와의 cross entropy loss 계산
        # 이 loss를 배치 당 loss로 보관
        loss = criterion(output, target)
        batch_losses.append(loss)

        # 기울기 계산
        loss.backward()

        # 가중치 업데이트!
        optimizer.step()
        
    # 배치당 평균 loss 계산
    avg_loss = sum(batch_losses) / len(batch_losses)
    
    return avg_loss

In [ ]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()

    batch_losses = []
    correct = 0 

    with torch.no_grad(): 
        for data, target in test_loader:
            # 예측값 생성
            output = model(data)

            # loss 계산 (이전과 동일)
            loss = criterion(output, target)
            batch_losses.append(loss)

           # Accuracy 계산
           # y pred와 y가 일치하면 correct에 1을 더해주기
            pred = output.max(1, keepdim=True)[1]

            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    # 배치 당 평균 loss 계산 
    avg_loss =  sum(batch_losses) / len(batch_losses)

    #정확도 계산
    accuracy = 100. * correct / len(test_loader.dataset)

    return avg_loss, accuracy

In [ ]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'[{epoch}] Train Loss: {train_loss:.4f}\tTest Loss: {test_loss:.4f}\tAccuracy: {test_accuracy:.2f}%')

[1] Train Loss: 0.8467	Test Loss: 0.3062	Accuracy: 90.71%
[2] Train Loss: 0.2365	Test Loss: 0.2033	Accuracy: 93.57%
[3] Train Loss: 0.1640	Test Loss: 0.1803	Accuracy: 94.27%
[4] Train Loss: 0.1315	Test Loss: 0.1520	Accuracy: 94.87%
[5] Train Loss: 0.1096	Test Loss: 0.1075	Accuracy: 96.73%
[6] Train Loss: 0.0904	Test Loss: 0.0976	Accuracy: 96.78%
[7] Train Loss: 0.0788	Test Loss: 0.2918	Accuracy: 90.84%
[8] Train Loss: 0.0837	Test Loss: 0.1005	Accuracy: 96.96%
[9] Train Loss: 0.0605	Test Loss: 0.0919	Accuracy: 97.16%
[10] Train Loss: 0.0530	Test Loss: 0.0883	Accuracy: 97.13%
[11] Train Loss: 0.0484	Test Loss: 0.1813	Accuracy: 94.21%
[12] Train Loss: 0.1624	Test Loss: 0.1175	Accuracy: 96.40%
[13] Train Loss: 0.0522	Test Loss: 0.1238	Accuracy: 96.06%
[14] Train Loss: 0.1043	Test Loss: 0.0844	Accuracy: 97.42%
[15] Train Loss: 0.0436	Test Loss: 0.0740	Accuracy: 97.55%
[16] Train Loss: 0.0371	Test Loss: 0.0753	Accuracy: 97.71%
[17] Train Loss: 0.0323	Test Loss: 0.0787	Accuracy: 97.54%
[18] T

## 과제 4
과제 3 부분의 성능을 지금까지 배운 지식을 바탕으로 향상시켜보세요

- Hint : Activation function, hyperparameter setting

In [ ]:
# Assignment 4 구현은 여기서 ()
# optimizer Adam , lr 수정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [ ]:
EPOCHS = 15

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'[{epoch}] Train Loss: {train_loss:.4f}\tTest Loss: {test_loss:.4f}\tAccuracy: {test_accuracy:.2f}%')

[1] Train Loss: 7.9618	Test Loss: 0.4892	Accuracy: 86.80%
[2] Train Loss: 0.4161	Test Loss: 0.3805	Accuracy: 89.24%
[3] Train Loss: 0.3332	Test Loss: 0.3107	Accuracy: 91.41%
[4] Train Loss: 0.2962	Test Loss: 0.3020	Accuracy: 91.26%
[5] Train Loss: 0.2754	Test Loss: 0.3250	Accuracy: 91.31%
[6] Train Loss: 0.2715	Test Loss: 0.3346	Accuracy: 90.92%
[7] Train Loss: 0.2561	Test Loss: 0.2855	Accuracy: 91.86%
[8] Train Loss: 0.2509	Test Loss: 0.2843	Accuracy: 92.17%
[9] Train Loss: 0.2425	Test Loss: 0.2846	Accuracy: 92.08%
[10] Train Loss: 0.2494	Test Loss: 0.3026	Accuracy: 91.88%
[11] Train Loss: 0.2410	Test Loss: 0.3169	Accuracy: 91.88%
[12] Train Loss: 0.2295	Test Loss: 0.2725	Accuracy: 92.98%
[13] Train Loss: 0.2218	Test Loss: 0.2666	Accuracy: 92.99%
[14] Train Loss: 0.2221	Test Loss: 0.2679	Accuracy: 92.63%
[15] Train Loss: 0.2148	Test Loss: 0.2441	Accuracy: 94.01%


In [ ]:
# hidden layer의 node 수 변경

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(784,256)
        self.layer2 = nn.Linear(256,128)
        self.layer3 = nn.Linear(128,10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)

        return out

In [ ]:
model = Net()
model

Net(
  (layer1): Linear(in_features=784, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [ ]:
EPOCHS = 15

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'[{epoch}] Train Loss: {train_loss:.4f}\tTest Loss: {test_loss:.4f}\tAccuracy: {test_accuracy:.2f}%')

[1] Train Loss: 0.8936	Test Loss: 0.3561	Accuracy: 88.57%
[2] Train Loss: 0.2375	Test Loss: 0.2120	Accuracy: 93.53%
[3] Train Loss: 0.1647	Test Loss: 0.1870	Accuracy: 93.89%
[4] Train Loss: 0.1284	Test Loss: 0.2238	Accuracy: 92.64%
[5] Train Loss: 0.1048	Test Loss: 0.1300	Accuracy: 95.96%
[6] Train Loss: 0.0875	Test Loss: 0.1049	Accuracy: 96.77%
[7] Train Loss: 0.0757	Test Loss: 0.1162	Accuracy: 96.09%
[8] Train Loss: 0.0643	Test Loss: 0.0946	Accuracy: 97.00%
[9] Train Loss: 0.0560	Test Loss: 0.0829	Accuracy: 97.54%
[10] Train Loss: 0.0489	Test Loss: 0.1038	Accuracy: 96.65%
[11] Train Loss: 0.0433	Test Loss: 0.0759	Accuracy: 97.63%
[12] Train Loss: 0.0373	Test Loss: 0.0927	Accuracy: 96.95%
[13] Train Loss: 0.0337	Test Loss: 0.0670	Accuracy: 98.02%
[14] Train Loss: 0.0289	Test Loss: 0.2166	Accuracy: 93.66%
[15] Train Loss: 0.1717	Test Loss: 0.1339	Accuracy: 95.77%


In [ ]:
model = Net()
model

Net(
  (layer1): Linear(in_features=784, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
)

**무엇을 보완하였고, 왜 보완되었는지에 대한 자유 서술 (아래에)**

In [ ]:
# 첫번째로 optimizer와 learning rate 수정하였으나 가장 처음 것보다 좋지 않았다.
# 두번째로 second hidden layer의 node 수를 128개로 늘리고 나머지는 그대로 유지하였는데 
# loss가 더 빨리 줄어들면서 약간 더 좋은 성능을 보였다. 
# parameter수가 조금 늘어나서 복잡한 모델이 만들어졌기 때문이라고 생각한다.